# Creating random text data for Pandas

## Creating random length sentences of random length words using random characters

### Background

A friend recently asked about transforming data in a Pandas data frame and provided an example of what the target column contains currently (prior to the transformation).

After I and a few others made some suggestions, there was also some discussion on performance of the different method of applying the desired transformation.

So that we could properly test, I decided to make some test data, and this in itself provided a challenge.

Briefly, the column of data contained JSON data with:

1. A single-element array
2. Containing an Object (actually, more like a set in Python)
3. Containing one or more words separated by commas

An example (similar to that provided by my friend):

`[{These, words, are, contained, by, an, Object, or, set, which, itself, is, contained, by, an, array}]`  
`[{The, next, set, of, words, is, like, this}]`

As yet, it is not clear whether the data contained the individual words wrapped by double quotes or not, and possibly the data was actually

`[{"These", "words", "are", "contained", "by", "an", "Object", "or", "set", "which", "itself", "is", "contained", "by", "an", "array"}]`  
`[{"The", "next", "set", "of", "words", "is", "like", "this"}]`

Having this example, the challenge is to create a lot of data with a random number of words in each "sentence", where each word contains a random number of letters.

Just the notion of the word "random" implies using `numpy.random` however, none of the methods available in that class can help us directly in our task. For that, we will also need to make use of `itertools` and a not so well known (well, at east to me) method of splitting an array by unequal length intervals using `numpy.split`.

In [1]:
import pandas as pd
from numpy import random as rand
import numpy as np
import itertools as itools
#import unicodedata
import re

In [2]:
NUM_RECORDS = 1000000
MAX_WORDS_PER_SENTENCE = 20
MAX_CHARS_PER_WORD = 10

In [3]:
# Create a "template" (of sorts) for our records.
# This template will tell us how many words randomly will be in each record.
sentence_templates = rand.randint(low=1, high=MAX_WORDS_PER_SENTENCE+1, size=NUM_RECORDS)

# Create another "template" which will be a random number of characters for each word.
# We know the count of all words across all records by using the sum() method of the numpy array sentence_templates
word_templates = rand.randint(low=1, high=MAX_CHARS_PER_WORD+1, size=sentence_templates.sum())

In [4]:
# Let's just see how many words we will have in total
len(word_templates)

10513416

In [5]:
# Now use itertools to creating a running count of words
# For example, if the first 5 sentences (records) had word counts of 7, 3, 4, 8, 2
# itertools would return 7, 10, 14, 22, 24
# This running count is needed in the numpy.split method later
words_iter = list(itools.accumulate(word_templates))

In [6]:
# Now let's create an array of random characters from A-Z,
# having length = the last element of our word iterator.
# Similar to the example of 5 above where 24 would be returned as the size.
asc_codes = rand.randint(low=65, high=91, size=words_iter[-1:])

In [7]:
# Let's take a peek at the ascii codes that were generated
asc_codes[:50]

array([72, 84, 65, 90, 74, 88, 71, 78, 83, 75, 74, 66, 89, 71, 72, 71, 73,
       66, 76, 72, 90, 80, 87, 79, 82, 74, 80, 77, 78, 79, 75, 79, 85, 80,
       73, 86, 73, 75, 68, 89, 88, 68, 81, 68, 82, 84, 81, 70, 87, 80])

In [8]:
#f = lambda x: chr(x)
#chars = f(asc_codes.tolist())
# The above did not work, so we will use the following instead
# to convert the ascii codes to characters
chars = [chr(x) for x in asc_codes.tolist()]
chars[:20]

['H',
 'T',
 'A',
 'Z',
 'J',
 'X',
 'G',
 'N',
 'S',
 'K',
 'J',
 'B',
 'Y',
 'G',
 'H',
 'G',
 'I',
 'B',
 'L',
 'H']

In [9]:
# Taking the long list of characters that was generated by our conversion from ascii to char
# we use the numpy.split() method in which the 2nd parameter tells us at which index locations
# to split the array (this is why ithertools was handy)
words = [''.join(x) for x in np.split(chars, words_iter)]

# Let's see the first 20 words we created
words[:20]

['HTAZJ',
 'XGNSKJBYGH',
 'GIBLHZPWOR',
 'JPM',
 'NO',
 'KOUPIVIKD',
 'YXDQDRT',
 'QFWPDAMTE',
 'TFNTIXY',
 'LJKLH',
 'H',
 'JXOIGIXY',
 'LQRSU',
 'DTFTOSYQI',
 'B',
 'OWYTLDQ',
 'JFVDHYOISG',
 'SBDZWKS',
 'LM',
 'GZZIM']

In [10]:
# At this point, we have a list of words but we still need to split those words up into sentences (or records)
# Once again, we use itertools to accumulate the count so that we can pass this to the numpy.split() method
sentences_iter = list(itools.accumulate(sentence_templates))

sentences_iter[:20]

[20,
 22,
 34,
 43,
 44,
 63,
 72,
 85,
 91,
 94,
 105,
 120,
 133,
 145,
 150,
 163,
 180,
 188,
 196,
 211]

In [11]:
# Here we apply the numpy.split method, but this time to a list of words rather than a list of characters

# For the source data, if we want the individual words to be wrapped in quotes, use:
#sentences = ['[{"' + str('", "'. join(x)) + '"}]' for x in np.split(words, sentences_iter)]
# For the source data, if we want no quotes around the individual words, use:
sentences = ['[{' + str(', '. join(x)) + '}]' for x in np.split(words, sentences_iter)]
sentences[:10]

['[{HTAZJ, XGNSKJBYGH, GIBLHZPWOR, JPM, NO, KOUPIVIKD, YXDQDRT, QFWPDAMTE, TFNTIXY, LJKLH, H, JXOIGIXY, LQRSU, DTFTOSYQI, B, OWYTLDQ, JFVDHYOISG, SBDZWKS, LM, GZZIM}]',
 '[{ILJK, FJPUS}]',
 '[{EN, YMIDL, B, U, KLWEWCXXIY, TNZNSKP, NXJFRZSWTC, TWMSL, I, PXR, UYMDVC, QXVPYXRPH}]',
 '[{NCW, NFVEDSXKZD, ERJOVTZTCM, MMBVQD, MEGWZXK, IXAKQ, IL, XEAMSA, DH}]',
 '[{VIZBOP}]',
 '[{NVPS, RFQDMTFUHS, EF, QHVBTNWFM, RI, MWFIECKRJ, WPJXNK, PQVKBFLL, LS, IDDVCQTXWW, MBZMV, VEAQG, TID, JLSWEOZHQ, BEJFOZEO, ZB, KUISMIVAS, OQOKS, DUKL}]',
 '[{POSUXULJY, MFZOATJ, POWKXUONC, NR, MRPONJVE, KWDLJVQ, MIKOG, WCSZDGEC, UZYOXAVRS}]',
 '[{ZRRKN, GVHJNYJ, PQCQFU, DPQQ, AW, QVZXJTT, RISZUFAU, LGTMJ, ZI, DCATPH, OB, KETD, EOJMICQ}]',
 '[{VPNAS, HTKNOXUDKT, D, EXHMXPF, TYHTGKPET, FBVFSSW}]',
 '[{HAZYHOQ, FFLYRIKRKN, EBUMR}]']

In [12]:
# Clean up the data - we typically have an empty "sentence" as the last one
if sentences[-1:] == ['[{}]']:
    sentences = sentences[:-1]

In [13]:
# Now that we have a ist of "sentences", we can push them into a Pandas data frame
df_sentences = pd.DataFrame(sentences, columns=["JSON_words"])
df_sentences.head()

,JSON_words
0,"[{HTAZJ, XGNSKJBYGH, GIBLHZPWOR, JPM, NO, KOUP..."
1,"[{ILJK, FJPUS}]"
2,"[{EN, YMIDL, B, U, KLWEWCXXIY, TNZNSKP, NXJFRZ..."
3,"[{NCW, NFVEDSXKZD, ERJOVTZTCM, MMBVQD, MEGWZXK..."
4,[{VIZBOP}]


In [14]:
# And now, we can save the data frame so that
# we can repeatedly test against the same data using different methods
df_sentences.to_csv("create_random_words_test_data.csv", sep="|", index=False)

In [15]:
# Read the data back from the file we saved, and re-display the head
df_map = pd.read_csv("create_random_words_test_data.csv", sep="|")
df_map.head()

,JSON_words
0,"[{HTAZJ, XGNSKJBYGH, GIBLHZPWOR, JPM, NO, KOUP..."
1,"[{ILJK, FJPUS}]"
2,"[{EN, YMIDL, B, U, KLWEWCXXIY, TNZNSKP, NXJFRZ..."
3,"[{NCW, NFVEDSXKZD, ERJOVTZTCM, MMBVQD, MEGWZXK..."
4,[{VIZBOP}]


In [16]:
# Apply one of the transformations as an example

# If the source data does not contain words wrapped with double quotes, use:
df_map["JSON_words"] = df_map["JSON_words"].map(lambda x: ['"' + y + '"' for y in re.split("[, ]+", x[2:-2])])
# If the source data already contains words wrapped with double quotes, use:
#df_map["JSON_words"] = [re.sub("[{}]", "", str(x)) for x in df_map["JSON_words"]]

In [17]:
df_map.head()

,JSON_words
0,"[""HTAZJ"", ""XGNSKJBYGH"", ""GIBLHZPWOR"", ""JPM"", ""..."
1,"[""ILJK"", ""FJPUS""]"
2,"[""EN"", ""YMIDL"", ""B"", ""U"", ""KLWEWCXXIY"", ""TNZNS..."
3,"[""NCW"", ""NFVEDSXKZD"", ""ERJOVTZTCM"", ""MMBVQD"", ..."
4,"[""VIZBOP""]"
